In [ ]:
import re
import os
import csv
import json
import time
import httpx
import asyncio
import requests
from urllib import parse
from bs4 import BeautifulSoup
from library.output_data import exportData

In [ ]:
# header list
header_list = [
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0'},
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'},
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/114.0'},
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 OPR/100.0.0.0'},
    {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
]

active_header = header_list[0]

### this is the phase-1 program that will collect thread pages    

In [ ]:
# urls for finding the the pages of any thread pages

# url_list = [
#     "https://www.forexfactory.com/thread/145348-need-help-and-explanation-on-interpretation-of-news",
#     "https://www.forexfactory.com/thread/9809-how-to-become-emotionless",
#     "https://www.forexfactory.com/thread/1228091-from-learning-through-losses-to-achievement",
#     "https://www.forexfactory.com/thread/197341-do-you-use-stoploss"
# ]

In [ ]:
# function to get the html

async def get_html(client, url, header):
    res = await client.get(url, headers=header)
    
    if res.status_code == 200:
        return {"result": res.status_code, "url": res.url, "html_string": res.text}
    else:
        return {"result": res.status_code, "url": res.url, "html_string": None}

In [ ]:
# the get_url function for collecting all the page urls for a threads

async def get_url(header, url_list):
    
    # url list to send request on ----for test----
    # url_list = [
    #     "https://www.forexfactory.com/thread/145348-need-help-and-explanation-on-interpretation-of-news",
    #     "https://www.forexfactory.com/thread/9809-how-to-become-emotionless",
    #     "https://www.forexfactory.com/thread/1228091-from-learning-through-losses-to-achievement",
    #     "https://www.forexfactory.com/thread/197341-do-you-use-stoploss"
    # ]

    # getting the list data
    html_response_list = []
    error_response_list = []
    post_url_list = []

    async with httpx.AsyncClient(timeout=None) as client:
        tasks = []

        # appending async task to tasks
        for url in url_list:
            tasks.append(asyncio.ensure_future(get_html(client, url["thread_url"], header)))

        # printing the number of requests
        print(f"total number of request send: {len(url_list)}")

        # waiting to collect all the response
        async_response = await asyncio.gather(*tasks)

        # getting the thead_list data
        thread_url_list = []
        error_response_list = []

        for response in async_response:
            if response['result'] == 200:
                html_response = {
                    "html_string": response["html_string"],
                    "url": response["url"]
                }

                html_response_list.append(html_response)
            else:
                error_response_list.append(response)


        # parsing and finding all the thread apge url
        for html_response in html_response_list:
            html = html_response["html_string"]
            url = str(html_response["url"])
            parsed_url = parse.urlparse(url)
            # print(parsed_url)

            soup = BeautifulSoup(html, 'lxml')
            page_num_tags = soup.select('a[title="Last Page"]')

            if page_num_tags:
                for page_tag in page_num_tags:
                    if page_tag.string is not None:
                        page_num = page_tag.string
                        break
            else:
                page_num = 1

            # print(page_num)

            if int(page_num) > 1:
                query_list = [f"page={page}" for page in range(1, int(page_num) + 1)]
                # generating all the urls
                for custom_query in query_list:
                    post_url = parsed_url._replace(query=custom_query).geturl()
                    post_url_list.append(post_url)
            else:
                post_url = url
                post_url_list.append(post_url)


    return post_url_list

                        

In [ ]:
# getting all user input
# reading files from local directory
# file_path: threads_data/json/interactive_trading.json,
# file_path: threads_data/json/commercial_content.json, ----
# file_path: threads_data/json/trading_journals.json, ----
# file_path: threads_data/json/platform_tech.json,
# file_path: threads_data/json/trading_systems.json, ----
# file_path: threads_data/json/rookie_talk.json, ----
# file_path: threads_data/json/broker_discussion.json, ----
# file_path: threads_data/json/trading_discussion.json ----


file_dir = input('Enter file directory: ')
file_name = input('Enter the file_name: ')

In [ ]:
# getting the complete url list
with open(file_dir, mode='r') as json_file:
    json_data = json_file.read()
    url_list = json.loads(json_data)
    print(f"total number of threads: {len(url_list)}")
    print(url_list[-1])

In [ ]:
## running the concurrent
start_time = time.perf_counter()
post_url_list = await get_url(active_header, url_list) # running the crawler
stop_time = time.perf_counter()

print(len(post_url_list))

In [ ]:
save_dir = f"post_data/{file_name}_post_urls.json"

with open(save_dir, mode='w') as json_file:
    json_data = json.dumps(post_url_list)
    json_file.write(json_data)
    print('Saved the file!')